In [2]:
import pandas as pd
import numpy as np
import scipy

In [4]:
#tumor stage and treatment_type
Liu_clinical=pd.read_csv('Liu_data/Liu_clinical.csv')
#because we don't know the tumor stage in dataset liu, we assign the tumor size 1.4e7, which is the median size of tumor in tcga

Liu_clinical['tumor_size']=1.400000e+07
Liu_clinical_sub=Liu_clinical[['Patient','tumor_size','OS','BR','cyclesOnTherapy']]

In [5]:
#ssGSEA of MHC-I and MHC-II
Liu_mhc=pd.read_csv('Liu_data/Liu_mhc.csv')


In [6]:
#abstract the pd-1 expression 
#Liu_genes=pd.read_csv('data/Liu_genes.csv')
# PDCD1=Liu_genes[Liu_genes['Unnamed: 0']=='PDCD1'].T
# PDCD1=PDCD1.reset_index()
# PDCD1.to_csv('data/Liu_PDCD1.csv')
Liu_pd=pd.read_csv('Liu_data/Liu_PDCD1.csv')

In [6]:
#from the cibersort analysis import the information of treg
#from the time signature import the informaiton of cd8+T 
Liu_time=pd.read_csv('Liu_data/Liu_TIME.csv')
Liu_time_sub=Liu_time[['Patient','SC T.CD8','T cells regulatory (Tregs)','T cells CD8']]
Liu_time_sub.head(3)

,Patient,SC T.CD8,T cells regulatory (Tregs),T cells CD8
0,Patient145,0.542359,0.0,0.313061
1,Patient125,2.685226,0.0,2.367949
2,Patient100,1.893131,0.0,2.114949


In [8]:
#merge all the table above
Liu_all=pd.merge(Liu_clinical_sub,Liu_mhc,on='Patient',how='inner')

In [9]:
Liu_all=pd.merge(Liu_all,Liu_pd,on='Patient',how='inner')

In [10]:
Liu_all=pd.merge(Liu_all,Liu_time_sub,on='Patient',how='inner')

In [11]:
#calculate the ratio of Treg to CD8+T in cibersort result
Liu_all['Treg_pro']=Liu_all['T cells regulatory (Tregs)']/Liu_all['T cells CD8']

In [12]:
#deal with abnormal value of Treg_pro
for i in range(len(Liu_all)):
    Treg_pro=Liu_all.loc[i,'Treg_pro']
    if Treg_pro>=0 and Treg_pro<100:
        pass
    else:
        Liu_all.loc[i,'Treg_pro']=0

In [13]:
#calculate total MHC expression in ssGSEA result
Liu_all['mhc']=Liu_all['MHC-I']+Liu_all['MHC-II']

In [14]:
x=Liu_all['mhc'].tolist()
y=Liu_all['SC T.CD8']

In [15]:
#correlation of MHC expression and CD8+T cells' scores
r = np.corrcoef(x, y)
scipy.stats.pearsonr(x, y) 

PearsonRResult(statistic=0.6742659420513787, pvalue=3.1374521291902884e-17)

In [16]:
#Because the expression of PDCD1 is long tail,normalize it by np.log(x+1)/np.log(max(x)+1)
Liu_all['PDCD1'].describe()
Liu_all['mu']=np.log(Liu_all['PDCD1']+1)/8.83

In [17]:
#normalize the MHC expression
Liu_all['mhc_norm']=(Liu_all['mhc']+22245)/(22210+22245)

In [18]:
Liu_all['SC T.CD8'].describe()

count    120.000000
mean       0.034480
std        0.592138
min       -1.059181
25%       -0.401139
50%       -0.072303
75%        0.341287
max        2.685226
Name: SC T.CD8, dtype: float64

In [19]:
cd8t_std=0.03/(2.231578+3.384689)
cd8t_mean=cd8t_std*2.231578
Liu_all['tcd8_pop']=(Liu_all['SC T.CD8']*cd8t_std+cd8t_mean)

In [20]:
Liu_all['tcd8_pop'].describe()

count    120.000000
mean       0.012104
std        0.003163
min        0.006263
25%        0.009778
50%        0.011534
75%        0.013743
max        0.026264
Name: tcd8_pop, dtype: float64

In [21]:
Liu_all.to_csv('Liu_all.csv')